<a href="https://colab.research.google.com/github/JialinLiOSU/ML-Final-Project/blob/master/transfer_learning_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import os
import time
import keras
# from vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
# from imagenet_utils import decode_predictions
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection  import train_test_split

In [16]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
# Loading the training data
# PATH = os.getcwd()
# Define data path
data_path = '/content/drive/My Drive/Colab Notebooks/region map'
data_dir_list = os.listdir(data_path)     
print(data_dir_list)

['us', 'world', 'south_korea', 'china']


In [0]:
def VGG_19(weights_path=None, input_tensor=None):
    img_input = input_tensor
# Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='last_layer')(x)


    return Model(img_input, x, name='vgg19')



In [24]:
img_data_list=[]

for dataset in data_dir_list:
	img_list=os.listdir(data_path+'/'+ dataset)
	print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
	for img in img_list:
		img_path = data_path + '/'+ dataset + '/'+ img
		img = image.load_img(img_path, target_size=(224, 224))
		x = image.img_to_array(img)
		x = np.expand_dims(x, axis=0)
		x = preprocess_input(x)
#		x = x/255
# 		print('Input image shape:', x.shape)
		img_data_list.append(x)

Loaded the images of dataset-us

Loaded the images of dataset-world

Loaded the images of dataset-south_korea

Loaded the images of dataset-china



In [25]:
img_data = np.array(img_data_list)
#img_data = img_data.astype('float32')
print (img_data.shape)
img_data=np.rollaxis(img_data,1,0)
print (img_data.shape)
img_data=img_data[0]
print (img_data.shape)

#Training the feature extraction also
batch_size=32
# epochs_list=[20,40,60,80,100]
epochs=100
layer_inx_list=[-3,-4,-5,-6,-7,-8,-9,-10]

# Define the number of classes
num_classes = 4
num_img_each_total=1100
num_img_each_list=[i for i in range(500,1100,100)] # 2

print(num_img_each_list)

(4009, 1, 224, 224, 3)
(1, 4009, 224, 224, 3)
(4009, 224, 224, 3)
[500, 600, 700, 800, 900, 1000]


In [0]:
import numpy as np
import os
import time
import keras
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection  import train_test_split
from keras import backend as K
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import cv2, numpy as np
from keras.layers import Activation, Conv2D
from keras.layers import Input
from keras.layers import merge
from keras.models import Model
from keras.optimizers import SGD
from scipy.misc import imread
from scipy.misc import imresize
from keras.utils import plot_model 

data_location = "/content/drive/My Drive/Colab Notebooks/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [31]:
for num_img_each in num_img_each_list:

    num_of_samples = num_img_each*num_classes
    labels = np.ones((num_of_samples,),dtype='int64')
    X1=img_data[0:num_img_each,]
    X2=img_data[num_img_each_total:num_img_each_total+num_img_each]
    X3=img_data[num_img_each_total*2:num_img_each_total*2+num_img_each]
    X4=img_data[num_img_each_total*3:num_img_each_total*3+num_img_each]
    X=np.concatenate((X1,X2,X3,X4),axis=0)

    labels[0:num_img_each]=0
    labels[num_img_each:num_img_each*2]=1
    labels[num_img_each*2:num_img_each*3]=2
    labels[num_img_each*3:num_img_each*4]=3
    # names = ['china','south_korea','us','world']
    # convert class labels to on-hot encoding
    Y = np_utils.to_categorical(labels, num_classes)
    #Shuffle the dataset
    x,y = shuffle(X,Y, random_state=2)
    # Split the dataset
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=2)
    print ("num_img_each:",num_img_each)

    for layer_inx in layer_inx_list:
  
        image_input = Input(shape=(224, 224, 3))
        model = VGG_19(weights_path=data_location, input_tensor=image_input)
        print ("layer_inx:",layer_inx)
        model.load_weights(data_location)

        last_layer = model.get_layer('last_layer').output
        x= Flatten(name='flatten')(last_layer)
        x = Dense(128, activation='relu', name='fc1')(x)
        x = Dense(128, activation='relu', name='fc2')(x)
        out = Dense(num_classes, activation='softmax', name='output')(x)
        custom_vgg_model2 = Model(image_input, out)

        # freeze all the layers except the dense layers
        for layer in custom_vgg_model2.layers[:layer_inx]:
            layer.trainable = False
            
        custom_vgg_model2.summary()
        
        custom_vgg_model2.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

        t=time.time()
        #	t = now()
        hist = custom_vgg_model2.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test))
        print('Training time: %s' % (time.time()-t))
        (loss, accuracy) = custom_vgg_model2.evaluate(X_test, y_test, batch_size=10, verbose=1)

        print()
        str1="Batch_size: "+str(batch_size)+' epochs: '+str(epochs)+'\n'
        str2="[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100)+'\n'
        str3='Training time: ' + str (time.time()-t)+'\n'
        str4='num_img_each: ' + str (num_img_each)+'\n'
        str5='layer_inx: ' + str (layer_inx)+'\n'
        data_path = '/content/drive/My Drive/Experiment results/'
        filename='Experiment_results_pretrained_big_training_size'+'.txt'
        filename=data_path+'Experiment_results_pretrained_big_training_size'+'.txt'
        file = open(filename,'a')
        file.write(str4) 
        file.write(str5)
        file.write(str1) 
        file.write(str2)
        file.write(str3)
        file.close()

num_img_each: 500
layer_inx: -3
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 

FileNotFoundError: ignored

In [0]:
data_path = '/content/drive/My Drive/Experiment results/'
str1="Batch_size: "
filename=data_path+'Experiment_results_pretrained_big_training_size'+'.txt'
file = open(filename,'a')
file.write(str1) 
file.close()